# 5 Insering Predicted Game Results on Database for App

## 5.1 Import the Libs

In [2]:
import pandas as pd
import requests
import lxml
from bs4 import BeautifulSoup
import time
import pyodbc
import numpy as np
from urllib.error import URLError, HTTPError
from urllib.request import Request, urlopen
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
import matplotlib.pyplot as plt

## 5.2 Reading tables from Database

In [3]:
# Function Select
def SQLSelect(query):
    conn = pyodbc.connect('Trusted_Connection=yes',
                      driver = '{ODBC Driver 17 for SQL Server}',
                      server = 'localhost',
                      database = 'MLS_Project')
    out = pd.read_sql_query(query,conn)
    return out

query_all_MatchDetails = '''
    Select 
    * 
    from TableMatchDetailsPredict
'''

def SQLTruncate(TableName):
    conn = pyodbc.connect('Trusted_Connection=yes',
                          driver = '{ODBC Driver 17 for SQL Server}',
                          server = 'localhost',
                          database = 'MLS_Project')

    cursor = conn.cursor()

    cursor.execute(f'''
    
                   TRUNCATE TABLE {TableName}
                     
                   ''')
    conn.commit()
    cursor.close()

In [4]:
df_MatchPred_v01 = SQLSelect(query_all_MatchDetails)

In [5]:
df_MatchPred_v01

,Generalkey,GAME_Date,TEAM_AWAY,TEAM_HOME,SEASON,Model_knn,Model_lr_mul,Model_sgdc,Model_svm,Model_svml,...,Model_rforest,Model_gauss,Model_adaboost,Model_gradboost,Pred_H,Pred_A,Pred_D,Pred_H_Perc,Pred_A_Perc,Pred_D_Perc
0,1,2023-02-25,St-Louis-City,Austin-FC,2023,H,H,H,H,H,...,H,H,H,H,10,0,0,1.0,0.0,0.0
1,2,2023-03-04,Charlotte-FC,St-Louis-City,2023,H,H,H,H,H,...,H,H,D,H,9,0,1,0.9,0.0,0.1
2,3,2023-03-11,St-Louis-City,Portland-Timbers,2023,H,H,H,H,H,...,H,H,H,H,10,0,0,1.0,0.0,0.0
3,4,2023-03-18,San-Jose-Earthquakes,St-Louis-City,2023,H,H,H,H,H,...,H,H,H,H,10,0,0,1.0,0.0,0.0
4,5,2023-03-25,St-Louis-City,Real-Salt-Lake,2023,H,H,H,H,H,...,H,H,A,H,9,1,0,0.9,0.1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,489,2023-04-29,Charlotte-FC,DC-United,2023,A,H,H,H,H,...,A,A,H,A,6,4,0,0.6,0.4,0.0
489,490,2023-05-31,CF-Montreal,DC-United,2023,A,H,A,H,H,...,A,A,A,A,3,7,0,0.3,0.7,0.0
490,491,2023-09-16,DC-United,Charlotte-FC,2023,H,H,H,H,H,...,H,H,A,H,8,2,0,0.8,0.2,0.0
491,492,2023-06-24,CF-Montreal,Charlotte-FC,2023,D,H,H,H,H,...,H,A,D,H,7,1,2,0.7,0.1,0.2


In [6]:
def SQLSelectMySQL(query):

    server = 'sql487.main-hosting.eu'
    database = 'u842004852_mlsproject_db'
    username = 'u842004852_mlsproject_use'
    password = 'vknP=j8O&a'
    driver = '{MySQL ODBC 8.0 ANSI Driver}'
    connection_string = f"DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}"

    conn = pyodbc.connect(connection_string)
    out = pd.read_sql_query(query, conn)
    conn.close()

    return out
# Needs to install MySQL driver on computer:
# https://dev.mysql.com/downloads/connector/odbc/

query_all_MATCH_PREDICTION = '''
    SELECT * 
    FROM MATCH_PREDICTION
'''

query_all_SCHEDULE = '''
    SELECT * 
    FROM SCHEDULE
    WHERE DATE(SCHEDULE_DATE) >= '2023-01-01' 
'''

query_all_TeamNames = '''
    SELECT * 
    FROM TEAMS
'''

In [8]:
df_MatchPred_v02_MySQL = SQLSelectMySQL(query_all_MATCH_PREDICTION)
df_MatchPred_v02_MySQL

,ID,H_TEAM_NAME,A_TEAM_NAME,PREDICTION,PRED_H_PERC,PRED_A_PERC,PRED_D_PERC


In [9]:
df_MatchSchedule_v01_MySQL = SQLSelectMySQL(query_all_SCHEDULE)
df_MatchSchedule_v01_MySQL.head(20)

,ID,SCHEDULE_DATE,H_TEAM_NAME,A_TEAM_NAME,SCHEDULE_TIME,SCORE,RESULT
0,3081,2023-02-25,Nashville,New York City,13:30:00,2:0,Nashville
1,3082,2023-02-25,Atlanta,San Jose,16:30:00,2:1,Atlanta
2,3083,2023-02-25,Charlotte,New England,16:30:00,0:1,New England
3,3084,2023-02-25,Cincinnati,Houston,16:30:00,2:1,Cincinnati
4,3085,2023-02-25,D.C.,Toronto,16:30:00,3:2,D.C.
5,3086,2023-02-25,Miami,Montréal,16:30:00,2:0,Miami
6,3087,2023-02-25,Orlando,New York,16:30:00,1:0,Orlando
7,3088,2023-02-25,Philadelphia,Columbus,16:30:00,4:1,Philadelphia
8,3089,2023-02-25,Austin,St. Louis,17:30:00,2:3,St. Louis
9,3090,2023-02-25,Dallas,Minnesota,17:30:00,0:1,Minnesota


## 5.2 Standarize Team Names

In [10]:
df_TeamNames_v01_MySQL = SQLSelectMySQL(query_all_TeamNames)
df_TeamNames_v01_MySQL

,TEAM_ID,TEAM_NAME,TEAM_SHORT_NAME,TEAM_ESPN,TEAM_ML
0,1,Atlanta United,Atlanta,Atlanta United FC,Atlanta-United
1,2,Austin FC,Austin,Austin FC,Austin-FC
2,3,Charlotte FC,Charlotte,Charlotte FC,Charlotte-FC
3,4,Chicago Fire FC,Chicago,Chicago Fire FC,Chicago-Fire
4,5,FC Cincinnati,Cincinnati,FC Cincinnati,FC-Cincinnati
5,6,Colorado Rapids,Colorado,Colorado Rapids,Colorado-Rapids
6,7,Columbus Crew,Columbus,Columbus Crew,Columbus-Crew
7,8,D.C. United,D.C.,D.C. United,DC-United
8,9,FC Dallas,Dallas,FC Dallas,FC-Dallas
9,10,Houston Dynamo FC,Houston,Houston Dynamo FC,Houston-Dynamo


In [11]:
TEAM_NAME_dict = pd.Series(df_TeamNames_v01_MySQL['TEAM_ML'].values,index=df_TeamNames_v01_MySQL['TEAM_SHORT_NAME']).to_dict()
TEAM_NAME_dict

{'Atlanta': 'Atlanta-United',
 'Austin': 'Austin-FC',
 'Charlotte': 'Charlotte-FC',
 'Chicago': 'Chicago-Fire',
 'Cincinnati': 'FC-Cincinnati',
 'Colorado': 'Colorado-Rapids',
 'Columbus': 'Columbus-Crew',
 'D.C.': 'DC-United',
 'Dallas': 'FC-Dallas',
 'Houston': 'Houston-Dynamo',
 'Kansas City': 'Sporting-KC',
 'LA': 'LA-Galaxy',
 'LAFC': 'Los-Angeles-FC',
 'Miami': 'Inter-Miami',
 'Minnesota': 'Minnesota-United',
 'Montréal': 'CF-Montreal',
 'Nashville': 'Nashville-SC',
 'New England': 'New-England-Revolution',
 'New York': 'New-York-Red-Bulls',
 'New York City': 'New-York-City-FC',
 'Orlando': 'Orlando-City',
 'Philadelphia': 'Philadelphia-Union',
 'Portland': 'Portland-Timbers',
 'Salt Lake': 'Real-Salt-Lake',
 'San Jose': 'San-Jose-Earthquakes',
 'Seattle': 'Seattle-Sounders-FC',
 'St. Louis': 'St-Louis-City',
 'Toronto': 'Toronto-FC',
 'Vancouver': 'Vancouver-Whitecaps-FC'}

In [12]:
df_MatchSchedule_v01_MySQL['TEAM_NAME_HOME'] = df_MatchSchedule_v01_MySQL['H_TEAM_NAME'].map(TEAM_NAME_dict)
df_MatchSchedule_v01_MySQL['TEAM_NAME_AWAY'] = df_MatchSchedule_v01_MySQL['A_TEAM_NAME'].map(TEAM_NAME_dict)


In [13]:
df_MatchSchedule_v01_MySQL

,ID,SCHEDULE_DATE,H_TEAM_NAME,A_TEAM_NAME,SCHEDULE_TIME,SCORE,RESULT,TEAM_NAME_HOME,TEAM_NAME_AWAY
0,3081,2023-02-25,Nashville,New York City,13:30:00,2:0,Nashville,Nashville-SC,New-York-City-FC
1,3082,2023-02-25,Atlanta,San Jose,16:30:00,2:1,Atlanta,Atlanta-United,San-Jose-Earthquakes
2,3083,2023-02-25,Charlotte,New England,16:30:00,0:1,New England,Charlotte-FC,New-England-Revolution
3,3084,2023-02-25,Cincinnati,Houston,16:30:00,2:1,Cincinnati,FC-Cincinnati,Houston-Dynamo
4,3085,2023-02-25,D.C.,Toronto,16:30:00,3:2,D.C.,DC-United,Toronto-FC
...,...,...,...,...,...,...,...,...,...
486,3569,2023-10-21,LA,Dallas,18:00:00,None,None,LA-Galaxy,FC-Dallas
487,3570,2023-10-21,Portland,Houston,18:00:00,None,None,Portland-Timbers,Houston-Dynamo
488,3571,2023-10-21,San Jose,Austin,18:00:00,None,None,San-Jose-Earthquakes,Austin-FC
489,3572,2023-10-21,St. Louis,Seattle,18:00:00,None,None,St-Louis-City,Seattle-Sounders-FC


## 5.4 Merge data: Table Prediction and Table Schedule

In [28]:
df_MatchPred_v02 = pd.merge(df_MatchPred_v01, 
                            df_MatchSchedule_v01_MySQL[['ID', 'SCHEDULE_DATE', 'TEAM_NAME_HOME', 'TEAM_NAME_AWAY']], 
                            left_on=['GAME_Date', 'TEAM_HOME', 'TEAM_AWAY'],
                            right_on=['SCHEDULE_DATE', 'TEAM_NAME_HOME', 'TEAM_NAME_AWAY'],
                            how='left')



In [29]:
na_list = df_MatchPred_v02['ID'].notnull()
df_MatchPred_v02[-na_list].iloc[:,[1,3,2,4]].sort_values(by=['GAME_Date'])



,GAME_Date,TEAM_HOME,TEAM_AWAY,SEASON
230,2023-02-27,Portland-Timbers,Sporting-KC,2023
27,2023-09-10,LA-Galaxy,St-Louis-City,2023


In [26]:
df_MatchSchedule_v01_MySQL[df_MatchSchedule_v01_MySQL['TEAM_NAME_HOME']=='New-York-City-FC']

,ID,SCHEDULE_DATE,H_TEAM_NAME,A_TEAM_NAME,SCHEDULE_TIME,SCORE,RESULT,TEAM_NAME_HOME,TEAM_NAME_AWAY
30,3113,2023-03-11,New York City,Miami,16:30:00,1:0,New York City,New-York-City-FC,Inter-Miami
45,3128,2023-03-18,New York City,D.C.,16:30:00,3:2,New York City,New-York-City-FC,DC-United
88,3171,2023-04-08,New York City,Atlanta,16:30:00,None,None,New-York-City-FC,Atlanta-United
100,3183,2023-04-15,New York City,Nashville,16:30:00,None,None,New-York-City-FC,Nashville-SC
114,3197,2023-04-22,New York City,Dallas,16:30:00,None,None,New-York-City-FC,FC-Dallas
195,3278,2023-05-27,New York City,Philadelphia,16:30:00,None,None,New-York-City-FC,Philadelphia-Union
211,3294,2023-05-31,New York City,Cincinnati,16:30:00,None,None,New-York-City-FC,FC-Cincinnati
221,3304,2023-06-03,New York City,New England,12:30:00,None,None,New-York-City-FC,New-England-Revolution
248,3331,2023-06-17,New York City,Columbus,12:30:00,None,None,New-York-City-FC,Columbus-Crew
300,3383,2023-07-05,New York City,Charlotte,16:30:00,None,None,New-York-City-FC,Charlotte-FC


In [27]:
from datetime import date
df_MatchPred_v01.loc[226,'GAME_Date'] = date(2023, 9, 2)



In [30]:
na_list = df_MatchPred_v02['ID'].notnull()
df_MatchPred_v03 = df_MatchPred_v02[na_list].iloc[:,[21,23,24,18,19,20]]
df_MatchPred_v03['ID'] = df_MatchPred_v03['ID'].astype(int)
df_MatchPred_v03

,ID,TEAM_NAME_HOME,TEAM_NAME_AWAY,Pred_H_Perc,Pred_A_Perc,Pred_D_Perc
0,3089,Austin-FC,St-Louis-City,1.0,0.0,0.0
1,3105,St-Louis-City,Charlotte-FC,0.9,0.0,0.1
2,3120,Portland-Timbers,St-Louis-City,1.0,0.0,0.0
3,3134,St-Louis-City,San-Jose-Earthquakes,1.0,0.0,0.0
4,3148,Real-Salt-Lake,St-Louis-City,0.9,0.1,0.0
...,...,...,...,...,...,...
488,3209,DC-United,Charlotte-FC,0.6,0.4,0.0
489,3292,DC-United,CF-Montreal,0.3,0.7,0.0
490,3480,Charlotte-FC,DC-United,0.8,0.2,0.0
491,3350,Charlotte-FC,CF-Montreal,0.7,0.1,0.2


In [31]:
df_MatchPred_v03.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 491 entries, 0 to 492
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              491 non-null    int32  
 1   TEAM_NAME_HOME  491 non-null    object 
 2   TEAM_NAME_AWAY  491 non-null    object 
 3   Pred_H_Perc     491 non-null    float64
 4   Pred_A_Perc     491 non-null    float64
 5   Pred_D_Perc     491 non-null    float64
dtypes: float64(3), int32(1), object(2)
memory usage: 24.9+ KB


In [32]:
df_MatchPred_v03 = df_MatchPred_v03.rename(columns={'ID': 'ID', 'TEAM_NAME_HOME': 'H_TEAM_NAME', 
                                                    'TEAM_NAME_AWAY': 'A_TEAM_NAME','Pred_H_Perc': 'PRED_H_PERC', 
                                                    'Pred_A_Perc': 'PRED_A_PERC','Pred_D_Perc': 'PRED_D_PERC'})
df_MatchPred_v03.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 491 entries, 0 to 492
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           491 non-null    int32  
 1   H_TEAM_NAME  491 non-null    object 
 2   A_TEAM_NAME  491 non-null    object 
 3   PRED_H_PERC  491 non-null    float64
 4   PRED_A_PERC  491 non-null    float64
 5   PRED_D_PERC  491 non-null    float64
dtypes: float64(3), int32(1), object(2)
memory usage: 24.9+ KB


## 5.5 Inserting Cleaned Table with Predictions into MySQL Database (App Database)

In [33]:
# Create a function to INSERT Table
def SQLInsertTablePred(TableGamePred_MySQL):
    base = pd.DataFrame(columns=['ID','H_TEAM_NAME','A_TEAM_NAME',
                                 'PREDICTION','PRED_H_PERC','PRED_A_PERC','PRED_D_PERC'])
    
    TableBase = base.append(TableGamePred_MySQL).fillna(0)

    # set up the connection string
    server = 'sql487.main-hosting.eu'
    database = 'u842004852_mlsproject_db'
    username = 'u842004852_mlsproject_use'
    password = 'vknP=j8O&a'
    driver = '{MySQL ODBC 8.0 ANSI Driver}'
    connection_string = f"DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}"
    
    conn = pyodbc.connect(connection_string)

    cursor = conn.cursor()

    # get the column names and data types from the dataframe
    columns = list(TableBase.columns)
    data_types = [str(TableBase[column].dtype) for column in columns]

    # create the SQL query
    query = f"INSERT INTO MATCH_PREDICTION ({', '.join(columns)}) VALUES ({', '.join(['?' for column in columns])})"
    
    
    for index, row in TableBase.iterrows():
        values = tuple(row)
        cursor.execute(query, values)

    
    conn.commit()
    conn.close()

In [34]:
SQLInsertTablePred(df_MatchPred_v03)